In [ ]:
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'  # always print last expr.

In [ ]:
import pyarrow as pa
from pyarrow.lib import ArrowNotImplementedError

i8 = pa.int8()
i64 = pa.int64()
i32 = pa.int32()
f64 = pa.float64()
td64 = pa.duration("s")
b = pa.bool_()

duration_arr = pa.array([-3, 2, -1, 1], type=td64)
int_arr = pa.array([-3, 2, -1, 1], type=i64)
float_arr = pa.array([-3, 2, -1, 1], type=f64)

# td64 = pa.int64()
# duration_arr = pa.array([-3, 2, -1, 1], type=i64)

unary_ops = [
    (pa.compute.negate, duration_arr, td64),
    (pa.compute.negate_checked, duration_arr, td64),
    (pa.compute.abs, duration_arr, td64),
    (pa.compute.abs_checked, duration_arr, td64),
    (pa.compute.sign, duration_arr, i8),
    # tests
    (pa.compute.is_null, duration_arr, b),
    (pa.compute.is_valid, duration_arr, b),
    (pa.compute.is_finite, duration_arr, b),
    (pa.compute.is_inf, duration_arr, b),
    (pa.compute.is_nan, duration_arr, b),
    (pa.compute.true_unless_null, duration_arr, b),
    # aggregations
    (pa.compute.min_max, duration_arr, pa.struct([('min', td64), ('max', td64)])),
    (pa.compute.max, duration_arr, td64),
    (pa.compute.min, duration_arr, td64),
    (pa.compute.sum, duration_arr, td64),
    (pa.compute.mode, duration_arr, pa.struct([('mode', td64), ('count', i64)])),
    # cumulative aggregations
    (pa.compute.cumulative_sum, duration_arr, td64),
    (pa.compute.cumulative_sum_checked, duration_arr, td64),
    (pa.compute.cumulative_min, duration_arr, td64),
    (pa.compute.cumulative_max, duration_arr, td64),
]

binary_ops = [
    # arithmetic
    (pa.compute.add, duration_arr, duration_arr, td64),
    (pa.compute.add_checked, duration_arr, duration_arr, td64),
    (pa.compute.subtract, duration_arr, duration_arr, td64),
    (pa.compute.subtract_checked, duration_arr, duration_arr, td64),
    (pa.compute.multiply, duration_arr, int_arr, td64),
    (pa.compute.multiply_checked, duration_arr, int_arr, td64),
    (pa.compute.divide, duration_arr, duration_arr, f64),
    (pa.compute.divide, duration_arr, int_arr, td64),
    (pa.compute.divide_checked, duration_arr, duration_arr, f64),
    (pa.compute.divide_checked, duration_arr, int_arr, td64),
    # comparisons
    (pa.compute.less, duration_arr, duration_arr, b),
    (pa.compute.less_equal, duration_arr, duration_arr, b),
    (pa.compute.greater, duration_arr, duration_arr, b),
    (pa.compute.greater_equal, duration_arr, duration_arr, b),
    (pa.compute.equal, duration_arr, duration_arr, b),
    (pa.compute.not_equal, duration_arr, duration_arr, b),
    # min/max
    (pa.compute.max_element_wise, duration_arr, duration_arr, td64),
    (pa.compute.min_element_wise, duration_arr, duration_arr, td64),
    # containment
    (pa.compute.is_in, duration_arr, duration_arr, b),
    (pa.compute.index_in, duration_arr, duration_arr, i32),
    # functions that require rounding
]

rounding_ops = [
    # operations that require rounding
    (pa.compute.mean, duration_arr, td64),
    (pa.compute.quantile, duration_arr, td64),
    (pa.compute.approximate_median, duration_arr, td64),
    (pa.compute.multiply, duration_arr, float_arr, td64),
    (pa.compute.multiply_checked, duration_arr, float_arr, td64),
    (pa.compute.divide, duration_arr, float_arr, td64),
    (pa.compute.divide_checked, duration_arr, float_arr, td64),
]

for op, *operands, dtype in unary_ops + binary_ops:
    try:
        result = op(*operands)
    except ArrowNotImplementedError as e:
        x = " "
    else:
        x = "x"
        assert result.type == dtype, f"{op}: got {result.type} expected {dtype}"

    formatted_ops = ", ".join(f"{op.type!s:<11}" for op in operands)
    print(f" [{x}] {op.__name__:<24}({formatted_ops}) -> {dtype}")

In [ ]:
binary_ops = [
    # arithmetic
    (pa.compute.add, duration_arr, duration_arr, td64),
    (pa.compute.add_checked, duration_arr, duration_arr, td64),
    (pa.compute.subtract, duration_arr, duration_arr, td64),
    (pa.compute.subtract_checked, duration_arr, duration_arr, td64),
    (pa.compute.multiply, duration_arr, int_arr, td64),
    (pa.compute.multiply, duration_arr, float_arr, td64),
    (pa.compute.multiply_checked, duration_arr, int_arr, td64),
    (pa.compute.multiply_checked, duration_arr, float_arr, td64),
    (pa.compute.divide, duration_arr, duration_arr, f64),
    (pa.compute.divide, duration_arr, int_arr, td64),
    (pa.compute.divide, duration_arr, float_arr, td64),
    (pa.compute.divide_checked, duration_arr, duration_arr, f64),
    (pa.compute.divide_checked, duration_arr, int_arr, td64),
    (pa.compute.divide_checked, duration_arr, float_arr, td64),
    # comparisons
    (pa.compute.less, duration_arr, duration_arr, b),
    (pa.compute.less_equal, duration_arr, duration_arr, b),
    (pa.compute.greater, duration_arr, duration_arr, b),
    (pa.compute.greater_equal, duration_arr, duration_arr, b),
    (pa.compute.equal, duration_arr, duration_arr, b),
    (pa.compute.not_equal, duration_arr, duration_arr, b),
    # min/max
    (pa.compute.max_element_wise, duration_arr, duration_arr, td64),
    (pa.compute.min_element_wise, duration_arr, duration_arr, td64),
    # containment
    (pa.compute.is_in, duration_arr, duration_arr, b),
    (pa.compute.index_in, duration_arr, duration_arr, i32),
]

for op, lhs, rhs, dtype in binary_ops:
    try:
        result = op(lhs, rhs)
    except ArrowNotImplementedError as e:
        print(f" [ ] {op.__name__:<20}({lhs.type!s:<11}, {rhs.type!s:<11}) -> {dtype}")
    else:
        assert result.type == dtype, f"{op}: got {result.type} expected {dtype}"
        print(f" [x] {op.__name__:<20}({lhs.type!s:<11}, {rhs.type!s:<11}) -> {dtype}")

In [ ]:
pa.compute.max_element_wise(duration_arr, pa.compute.multiply(2,duration_arr))

In [ ]:
pa.struct([('min', td64), ('max', td64)])

In [ ]:
pa.compute.min_max(duration_arr).type == pa.struct([('min', td64), ('max', td64)])

In [ ]:
pa.compute.is_in(duration_arr, duration_arr)

In [ ]:
import numpy as np


In [ ]:
td = np.random.rand(100) * 10 * np.timedelta64(1, "s")

In [ ]:
np.mode(td)

In [ ]:
pa.compute.approximate_median?